## Spotify analysis on Italy

In [6]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

import spotipy      
from spotipy.oauth2 import SpotifyClientCredentials

# Import Keys from the Dashboard client
from config import ckey, skey


In [7]:
# Setting up Spotify API info
cid = ckey
secret = skey

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Instructions
* TIP: This next section assumes that you have already downloaded the csv files with the Top200 charts for the country you are working on:

* Create a subfolder in the folder this notebook is located in called "input_files". Add the owid-covid-data.csv file there, you'll need that later. Then make another subfolder inside input_files called "spotify_top200_charts". Save the csv files you download there.
* Go to https://spotifycharts.com
* Choose the country you want to work on.
* Download Weekly Top200 charts for 2019 and 2020, 1 chart per month. We agreed as a group to download the chart from last week of each month, to keep things consistent. Save them in the "spotify_top200_charts" folder you set up.

In [8]:
# Create a Dataframe from weekly chart data

path= r"input_files/spotify_top200_charts/*.csv"

# Set up main dataframe with data from FIRST file in each folder
country_tracks_df = pd.read_csv(glob.glob(path)[0], header=1) 

# This line extracts the year and month from the file name, 
# and sets up new columns called 'Year' and 'Moth' with that info
string = str(glob.glob(path)[0])
year_month =  re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)                                         
country_tracks_df[['Year']] = year_month[0][0]
country_tracks_df[['Month']] = year_month[0][1]

for file in glob.glob(path)[1:]:
        temp_df = pd.read_csv(file, header=1)
        string = str(file) 
        year_month = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)
        
        # Print (year_month)
        temp_df[['Year']] = year_month[0][0]
        temp_df[['Month']] = year_month[0][1]
        country_tracks_df = pd.concat([country_tracks_df,temp_df])
             
country_tracks_df = country_tracks_df.sort_values(['Year','Month'])                                  
country_tracks_df       


,Position,Track Name,Artist,Streams,URL,Year,Month
0,1,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2885889,https://open.spotify.com/track/2fkVZYFv9hOejIs...,2018,12
1,2,Happy Birthday,Sfera Ebbasta,2182720,https://open.spotify.com/track/7Cw97917dvg5xm6...,2018,12
2,3,Torna a casa,Måneskin,1857579,https://open.spotify.com/track/3590AAEoqH50z4U...,2018,12
3,4,La fine del mondo,Anastasio,1836968,https://open.spotify.com/track/3FhWEh7VTAOYK4e...,2018,12
4,5,All I Want for Christmas Is You,Mariah Carey,1599867,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2018,12
...,...,...,...,...,...,...,...
195,196,È sempre bello,Coez,215634,https://open.spotify.com/track/6IdiuMw1FSAvU3e...,2021,01
196,197,BABY (feat. Rosa Chemical),BLOODY VINYL,215551,https://open.spotify.com/track/3jtMheZu2P0tbWz...,2021,01
197,198,Lacri-ma,Gazzelle,215159,https://open.spotify.com/track/0N6QZADGv1zv3Jb...,2021,01
198,199,Cyborg (feat. Geolier),Guè Pequeno,214811,https://open.spotify.com/track/54wRUYPMEHLttFd...,2021,01


'2019'

,Position,Track Name,Artist,Streams,URL
0,1,È sempre bello,Coez,3438470,https://open.spotify.com/track/18LXrvm337xqvFx...
1,2,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2571770,https://open.spotify.com/track/2fkVZYFv9hOejIs...
2,3,Calma - Remix,Pedro Capó,2051691,https://open.spotify.com/track/5iwz1NiezX7WWjn...
3,4,Happy Birthday,Sfera Ebbasta,1680645,https://open.spotify.com/track/7Cw97917dvg5xm6...
4,5,Holding out for You (feat. Zara Larsson),Fedez,1584324,https://open.spotify.com/track/4EZhkTdGDdlLoHI...
...,...,...,...,...,...
195,196,Imposible,Luis Fonsi,235055,https://open.spotify.com/track/3ao3OVxHlb3C08v...
196,197,Sister (Pastiglie),Chadia Rodriguez,234699,https://open.spotify.com/track/3PJFpY6SkQ0juJA...
197,198,God's Plan,Drake,233303,https://open.spotify.com/track/6DCZcSspjsKoFjz...
198,199,Centro (feat. Coez),MadMan,233119,https://open.spotify.com/track/6DGl9TtxYTpMFrm...


In [26]:
print(glob.glob(path)[0])


input_files/spotify_top200_charts/regional-it-weekly-2020-04-17--2020-04-24.csv


In [9]:
# Get Track ID's

# Set up list of tacks to iterate through 
track_names = country_tracks_df['Track Name'].to_list()   
# Empty list to record track IDs into                
track_ids = []                                                    
 # Heads up: with 4800 tracks to process, this takes a while 
for track in track_names:                                        
    song_results = sp.search(q=track, type='track', limit=1)      
    try:
        # Prevents program from blowing up - few tracks
        track_ids.append(song_results['tracks']['items'][0]['id']) 
        # Just to let you know it's working lack track ids
        print (f"{track} song ID : {song_results['tracks']['items'][0]['id']}")   
    except IndexError:                                            
        track_ids.append(np.nan)

t. Dermot Kennedy) song ID : 6ft4hAq6yde8jPZY2i5zLr
Whoopty song ID : 5vGLcdRuSbUhD8ScwsGSdA
VENT'ANNI song ID : 34r4voojt51Kk6ax5FJI1M
Cuore Nero - prod. Frenetik&Orang3 song ID : 0ROJX4yr64AlknCaX71JDQ
Chico (feat. Rose Villain & Luchè) song ID : 3Xhl7OqWAFgVxQKrWhyVVO
A Un Passo Dalla Luna song ID : 3YNcQUeVOpM3SDmwBeGfMK
MIA song ID : 116H0KvKr2Zl4RPuVBruDO
The Business song ID : 6f3Slt0GbA2bPZlz0aIFXN
Estate song ID : 18NxSIKcBgidhFiY37s9tC
M' Manc (con Geolier & Sfera Ebbasta) song ID : 0MLu1cFYL4ikdRTdasCQT9
$€ Freestyle song ID : 5BbdKBZO0TH0GhfxUfyhL9
guccy bag song ID : 755xh1CnKw8zg2m3Uv4TEX
Blue Jeans (feat. Calcutta) song ID : 3teW1cB7e8HMmtHUeWF7Je
WILDPIRATA (feat. Tedua) – prod. Garelli & Chryverde song ID : 6iFxX0SdHKxUhiyLW9PtCy
Hypnotized song ID : 3LN41NUdHkyNqQhi9gExMm
Crepe song ID : 6L671UzJPyAzbHEhaNhDUJ
Hollywood (feat. Diplo) song ID : 0WXU7I74NEN1DMLIT406Wx
Take You Dancing song ID : 59qrUpoplZxbIZxk6X0Bm3
Therefore I Am song ID : 54bFM56PmE4YLRnqpW6Tha
Barce

## Add an "output_files" folder to export 

In [10]:
# Add Track IDs to dataframe
# Add new column with track IDs
country_tracks_df['Track ID'] = track_ids                  

In [11]:
# Drop empty songs and export dataframe to csv to back it up

# Use .dropna() to remove rows with missing data
clean_country_tracks_df = country_tracks_df.dropna(how='any')    
# Back up to .csv       
clean_country_tracks_df.to_csv("output_files/1_tracks_with_track_ids.csv", index = False)  

In [12]:
clean_country_tracks_df

,Position,Track Name,Artist,Streams,URL,Year,Month,Track ID
0,1,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2885889,https://open.spotify.com/track/2fkVZYFv9hOejIs...,2018,12,2fkVZYFv9hOejIsLzZy8ad
1,2,Happy Birthday,Sfera Ebbasta,2182720,https://open.spotify.com/track/7Cw97917dvg5xm6...,2018,12,2bnTgPFD75vbq1AhxLJfMu
2,3,Torna a casa,Måneskin,1857579,https://open.spotify.com/track/3590AAEoqH50z4U...,2018,12,3590AAEoqH50z4UmhMIY85
3,4,La fine del mondo,Anastasio,1836968,https://open.spotify.com/track/3FhWEh7VTAOYK4e...,2018,12,3c5ZPBOMDO1HLHRrQEwpNW
4,5,All I Want for Christmas Is You,Mariah Carey,1599867,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2018,12,0bYg9bo50gSsH3LtXe2SQn
...,...,...,...,...,...,...,...,...
195,196,È sempre bello,Coez,215634,https://open.spotify.com/track/6IdiuMw1FSAvU3e...,2021,01,6IdiuMw1FSAvU3e6bgWQVX
196,197,BABY (feat. Rosa Chemical),BLOODY VINYL,215551,https://open.spotify.com/track/3jtMheZu2P0tbWz...,2021,01,3jtMheZu2P0tbWzAYYWhvS
197,198,Lacri-ma,Gazzelle,215159,https://open.spotify.com/track/0N6QZADGv1zv3Jb...,2021,01,0N6QZADGv1zv3JbVIX6ctx
198,199,Cyborg (feat. Geolier),Guè Pequeno,214811,https://open.spotify.com/track/54wRUYPMEHLttFd...,2021,01,54wRUYPMEHLttFdLImOlAt


In [13]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_track_ids_df = pd.read_csv("output_files/1_tracks_with_track_ids.csv")
country_track_ids_df

,Position,Track Name,Artist,Streams,URL,Year,Month,Track ID
0,1,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2885889,https://open.spotify.com/track/2fkVZYFv9hOejIs...,2018,12,2fkVZYFv9hOejIsLzZy8ad
1,2,Happy Birthday,Sfera Ebbasta,2182720,https://open.spotify.com/track/7Cw97917dvg5xm6...,2018,12,2bnTgPFD75vbq1AhxLJfMu
2,3,Torna a casa,Måneskin,1857579,https://open.spotify.com/track/3590AAEoqH50z4U...,2018,12,3590AAEoqH50z4UmhMIY85
3,4,La fine del mondo,Anastasio,1836968,https://open.spotify.com/track/3FhWEh7VTAOYK4e...,2018,12,3c5ZPBOMDO1HLHRrQEwpNW
4,5,All I Want for Christmas Is You,Mariah Carey,1599867,https://open.spotify.com/track/0bYg9bo50gSsH3L...,2018,12,0bYg9bo50gSsH3LtXe2SQn
...,...,...,...,...,...,...,...,...
21731,196,È sempre bello,Coez,215634,https://open.spotify.com/track/6IdiuMw1FSAvU3e...,2021,1,6IdiuMw1FSAvU3e6bgWQVX
21732,197,BABY (feat. Rosa Chemical),BLOODY VINYL,215551,https://open.spotify.com/track/3jtMheZu2P0tbWz...,2021,1,3jtMheZu2P0tbWzAYYWhvS
21733,198,Lacri-ma,Gazzelle,215159,https://open.spotify.com/track/0N6QZADGv1zv3Jb...,2021,1,0N6QZADGv1zv3JbVIX6ctx
21734,199,Cyborg (feat. Geolier),Guè Pequeno,214811,https://open.spotify.com/track/54wRUYPMEHLttFd...,2021,1,54wRUYPMEHLttFdLImOlAt


In [26]:
# Use API again to get audio features

danceability = []                                     # Set up empty lists to store data in
energy = []
valence = []
loudness = []
key = []
mode = []
speechiness = []
acousticness = []
instrumentalness =[]
liveness = []
tempo = []
time_signature = []


In [14]:

for track in country_track_ids_df['Track ID']:           # Heads up: this takes a long time
    
    try:
        feat_results = sp.audio_features([track])
        danceability.append(feat_results[0]['danceability'])
        energy.append(feat_results[0]['energy'])
        valence.append(feat_results[0]['valence'])
        loudness.append(feat_results[0]['loudness'])
        key.append(feat_results[0]['key'])
        mode.append(feat_results[0]['mode'])
        speechiness.append(feat_results[0]['speechiness'])
        acousticness.append(feat_results[0]['acousticness'])
        instrumentalness.append(feat_results[0]['instrumentalness'])
        liveness.append(feat_results[0]['liveness'])
        tempo.append(feat_results[0]['tempo'])
        time_signature.append(feat_results[0]['time_signature'])


        #print (f"{track} Valence Score: {feat_results[0]['valence']}")  # Just to let you see it working                                 
    
    except TypeError:                                 # Covers you in case there is missing data
        danceability.append(np.nan)
        energy.append(np.nan)
        valence.append(np.nan)
        loudness.append(np.nan)
        key.append(np.nan)
        mode.append(np.nan)
        speechiness.append(np.nan)
        acousticness.append(np.nan)
        instrumentalness.append(np.nan)
        liveness.append(np.nan)
        tempo.append(np.nan)
        time_signature.append(np.nan)


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
# Add audio features to dataframe

# Add new columns with audio features
country_track_ids_df['Danceability'] = danceability     
country_track_ids_df['Valence'] = energy
country_track_ids_df['Energy'] = valence
country_track_ids_df['Loudness'] = loudness     
country_track_ids_df['Key'] = key
country_track_ids_df['Mode'] = mode
country_track_ids_df['Speechiness'] = speechiness     
country_track_ids_df['Acousticness'] = acousticness
country_track_ids_df['Instrumentalness'] = instrumentalness
country_track_ids_df['Liveness'] = liveness     
country_track_ids_df['Tempo'] = tempo
country_track_ids_df['Time_Signature'] = time_signature

In [ ]:
country_track_ids_df

In [ ]:
# Back up dataframe again to .csv

 # Use .dropna() to remove rows with missing data
clean_country_track_ids_df = country_track_ids_df.dropna(how='any')     
#Back up the dataframe to csv again    
clean_country_track_ids_df.to_csv("output_files/2_tracks_with_audio_features.csv", index=False) 

clean_country_track_ids_df

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_tracks_data_df = pd.read_csv("output_files/2_tracks_with_audio_features.csv")

country_tracks_data_df.head()

## Stats
* Danceability: A value of 0.0 is least danceable and 1.0 is most danceable.

* Acousticness: A measure from 0.0 to 1.0 of whether the track is acoustic.

* Energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity.

* Instrumentalness: Predicts whether a track contains no vocals. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content.

* Liveness: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live.

* Loudness: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track. Values typical range between -60 and 0 db.

* Speechiness: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value.

* Tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

* Valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).



AttributeError: 'DataFrame' object has no attribute 'column'

In [ ]:
stats = country_tracks_data_df.describe()
stats

In [40]:
# Use groupby to get average valence of the 200 songs in each month

country_tracks_data_groupby = country_tracks_data_df.groupby(["Year", 'Month'], as_index=False)['Valence'].mean()

country_tracks_data_groupby

KeyError: 'Column not found: Valence'

In [18]:
# Set up some basic plt formatting configurations

plt.rc('font', size=12)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=20)

In [19]:
# Plot a comparison of 2019 2020 valence scores

# Set up lists to plot

valence_2019 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2019]

valence_2020 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2020]
valence_2020.drop('Year', inplace=True, axis=1)

fig= plt.figure(figsize=(12,8))                                      # Set up figure size
fig.suptitle('SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH')         # Set up main title

y_axis = valence_2019['Valence']
x_axis = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',                  # Set up x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.plot(x_axis, valence_2019['Valence'], label="2019 Average Valence", marker='o', color='darkred')  # Plot 2019
plt.plot(x_axis, valence_2020['Valence'], label="2020 Average Valence", marker='o', color='steelblue')  # Plot 2020
# Set up axis titles
plt.xlabel('Months')                                                 
plt.ylabel('Valence Score')
# Set up axis limits
plt.xlim(-0.75, len(x_axis)-0.25)                                    
plt.ylim(0, 1)
# Include the legend
plt.legend()                                                         

plt.show()

/Users/yazminrodriguez/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


KeyError: 'Valence'

In [ ]:
# Compare valence scores with covid infection rate

covid_df = pd.read_csv("input_files/owid-covid-data.csv")                # read the covid data file

country_covid_df = covid_df.loc[covid_df['location'] == 'New Zealand']   # Filter for country of your choice

country_covid_df.head()

In [ ]:
# Filter data for 2020, and add a 'month' column

country_covid__2020_df = country_covid_df[country_covid_df.date.str.contains(r'2020.*')]

country_covid__2020_df['Month'] = ''
country_covid__2020_df.head()


In [ ]:
# Extract the month from the 'date' column and add it to the new 'month' column, for sorting later

for index, row in country_covid__2020_df.iterrows():
    month, = re.findall(f"2020-(\d\d)-", row['date'])
    country_covid__2020_df.at[index, 'Month'] = int(month)  

country_covid__2020_df.head()

In [ ]:
# Create a groupby to get the sum of new cases in each month

country_covid__2020_groupby = country_covid__2020_df.groupby(['Month'], as_index=False)['new_cases'].sum()
country_covid__2020_groupby

In [ ]:
# TIP: This next section is to add missing months. In the case of New Zealand, there was no data for January
# For other countries, it might vary. Here's how I added January to the dataframe:

country_covid__2020_groupby.loc[-1] = [1, 0]             # This adds a 1 (month of January) in the first columne 
                                                         # index 0), in the last row of the dataframe. 

country_covid__2020_groupby.index = country_covid__2020_groupby.index + 1   # shifts the index
country_covid__2020_groupby = country_covid__2020_groupby.sort_index()      # sorts by index

country_covid__2020_groupby = country_covid__2020_groupby.rename(columns={"new_cases": "New Cases"})

country_covid__2020_groupby

In [ ]:
# Merge the dataframes into one nice comparison dataframe to scatter plot

country_covid_valence_df = pd.merge(valence_2020, country_covid__2020_groupby, on="Month")

country_covid_valence_df

In [ ]:
# Add a new cases per million column

country_polulation = 5002100  #TIP: This the population of New Zealand. Adjust for your country

country_covid_valence_df['New Cases Per Million'] = country_covid_valence_df['New Cases'] / country_polulation *1000000

country_covid_valence_df

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

fig, ax1 = plt.subplots(figsize=(12,8))                   # Set up subplot figure and size
fig.suptitle('NEW ZEALAND: SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH COMPARED TO NEW COVID CASES')

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',       # Set up shared x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax1.set_xlabel('Months')                

ax1.set_ylabel('Average Valence Scores')                  # Set up first plot 
ax1.set_ylim([.2, .8])

ax1_ydata = country_covid_valence_df['Valence']
ax1.plot(months, ax1_ydata, label="Average Valence Scores", marker='o', color='darkred')                       
                        
ax2 = ax1.twinx()                                         # Set up second plot 
ax2.set_ylabel('New Cases Per Month')
ax2_ydata = country_covid_valence_df['New Cases Per Million']
ax2.set_ylim([0, ax2_ydata.max()+20])
ax2.plot(months, ax2_ydata, label="New Covid Cases Per Million", marker='o', color='steelblue')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()


In [ ]:
# This is a mini function that adds a qualitative label to the correlation r score in the scatter plot

def r_label(r):
    abs_r = abs(r)
    if abs_r >= .8 : return "Very Strong"
    elif abs_r >= .6 : return "Strong"
    elif abs_r >= .4: return "Moderate"
    elif abs_r >= .2: return "Low"
    else: return "Negligible"

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

x_axis = country_covid_valence_df['Valence']                                 # Set up axes
y_axis = country_covid_valence_df['New Cases Per Million']

slope, intercept, rvalue, pvalue, stderr = stats.linregress(x_axis, y_axis)  # Get elements of regression equation
regress_values = x_axis * slope + intercept                                  # Calculate regression values

plt.figure(figsize=(12, 8))
plt.title('NEW ZEALAND: SPOTIFY LISTENER VALENCE PREFERENCE VS. NEW COVID CASES') # CHANGE TITLE TO REFLECT YOUR COUNTRY

plt.xlabel(f"New Covid Cases")                                               # Set x axis label for subplot                  
plt.ylabel(f"Valence")                                                       # Set title for subplot

r = round(stats.pearsonr(x_axis, y_axis)[0],2)                       # Calculate correlation coefficient
rlabel = r_label(r)                                                 # Call function to create a label for the r number

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))    # Ression formula

plt.scatter(x_axis, y_axis, facecolors="darkred",  alpha=.5,        # Plot the scatter chart
label=f"r = {r}\nCorrelation is {rlabel}\n{line_eq}" ) 

plt.plot(x_axis, regress_values, color="steelblue")                 # Plot the regression line    

plt.legend()                                                        # Add the legend
    
plt.savefig("output_files/valence_vs_newcases.png")                 # Save the png file   

plt.show()